## Import

In [45]:
import folium 
import pandas as pd
import shapely as shp
import geopandas as gpd
from sodapy import Socrata

## Ingest data

### Utility functions

In [41]:
def create_geometry(row, mappping):
    lat = row[mappping['wgs84_y']]
    lon = row[mappping['wgs84_x']]
    return shp.geometry.Point(lon, lat)

### Load dataframe

In [3]:
client = Socrata("www.dati.lombardia.it",
                 "UoUBbv9A1VT5Fxv6LxYu6LlU2",
                 username="m.scatassi@campus.unimib.it",
                 password="wSau!K6Ad!mi4fU")

In [104]:
results = client.get("mtva-9hrb", 
                     where="comune='MILANO'")

In [105]:
df = pd.DataFrame.from_records(results)

In [106]:
df.head(5)

,id,istat,provincia,comune,indirizzo,civico,data_attiv,data_aut,sup_alim,sup_non_alim,sup_tot,wgs84_y,wgs84_x,location
0,480452,15146,MILANO,MILANO,Piazza PREALPI,7,2013-05-29T00:00:00.000,2013-05-29T00:00:00.000,0,37,37,9.15249709999714,45.49438969887889,"{'human_address': '{""address"": ""45 49438969887..."
1,469652,15146,MILANO,MILANO,PIAZZALE LORETO,SNC,1998-04-20T00:00:00.000,1998-04-20T00:00:00.000,190,0,190,9.21495479999597,45.486261598879004,"{'human_address': '{""address"": ""45 48626159887..."
2,469692,15146,MILANO,MILANO,Via ANSPERTO,7,1998-08-26T00:00:00.000,1998-08-26T00:00:00.000,0,90,90,9.178966699996648,45.46470579887949,"{'human_address': '{""address"": ""45 46470579887..."
3,489239,15146,MILANO,MILANO,VIA BASSI UGO,5,2021-11-05T00:00:00.000,2021-11-05T00:00:00.000,0,125,125,9.184472699996537,45.48905239887895,"{'human_address': '{""address"": ""45 48905239887..."
4,470278,15146,MILANO,MILANO,Via BELFIORE,9,1999-10-25T00:00:00.000,1999-10-25T00:00:00.000,0,39,39,9.15784739999704,45.467346698879496,"{'human_address': '{""address"": ""45 46734669887..."


The fields:
- 'data_attiv'
- 'data_aut'

are the same, so we can drop one of them.

In [90]:
(df.data_attiv == df.data_aut).all()

True

In [91]:
df.drop('data_aut', axis=1, inplace=True)

The fields:
- sup_alim
- sup_non_alim
- sup_tot

are strings, so we need to convert them to numbers.

In [115]:
df.sup_alim = df.sup_alim.apply(lambda x: float(x))
df.sup_non_alim = df.sup_non_alim.apply(lambda x: float(x))
df.sup_tot = df.sup_tot.apply(lambda x: float(x))

We are interested only in the alimentary stores, so we can drop the other rows.

In [116]:
df = df[df.sup_alim > 0]

### Convert to gdf

In [117]:
mappping = {col: i for i, col in enumerate(df.columns)}

In [118]:
df["geometry"] = df.apply(create_geometry, mappping=mappping, axis=1)

In [127]:
gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf = gdf.set_crs(epsg=4326).reset_index(drop=True)

## Visualize data

In [128]:
center = shp.convex_hull(shp.geometry.MultiPoint(gdf.geometry)).centroid.coords[0]

In [131]:
map = folium.Map(location=center, zoom_start=12)
for point, sup_tot in zip(gdf.geometry, gdf.sup_tot):
    if sup_tot > 250:
        folium.Circle([point.x, point.y], 
                      color="red",
                      radius=sup_tot*0.1).add_to(map)
    else:
        folium.Circle([point.x, point.y],
                      color="blue").add_to(map)
    
map